In [13]:
import numpy as np
import pandas as pd
import random
from openpyxl import load_workbook

In [14]:
wb = load_workbook(filename= 'dataset.xlsx')
sheet_ranges = wb['Sheet 1']

df = pd.DataFrame(sheet_ranges.values)
df

col = df[1:72][[4,5,6,7,9,10,11,12,13]]
col.columns = ['UAW','Smpl','Avg','Cmplx','UUCW','TCF','ECF','UCP','Real_Effort']
cols = pd.DataFrame(col)

cols.insert(9,"Desired_UUCW", 0 , True)
cols

,UAW,Smpl,Avg,Cmplx,UUCW,TCF,ECF,UCP,Real_Effort,Desired_UUCW
1,9,6,10,15,355,0.81,0.84,398.5,7970,0
2,8,4,20,15,445,0.99,0.99,398.1,7962,0
3,9,1,5,20,355,1.03,0.8,396.75,7935,0
4,8,5,10,15,350,0.9,0.91,390.25,7805,0
5,8,1,10,16,345,0.9,0.91,387.9,7758,0
...,...,...,...,...,...,...,...,...,...,...
67,18,2,18,18,460,1.05,0.95,290,5800,0
68,17,1,23,22,565,1.03,0.8,289.55,5791,0
69,19,5,30,10,475,0.95,0.92,289.1,5782,0
70,6,5,15,5,250,1,0.92,288.9,5778,0


In [15]:
for index, row in cols.iterrows():
    UAW = cols.loc[index,"UAW"]
    UUCW = cols.loc[index,"UUCW"]
    TCF = cols.loc[index,"TCF"]
    ECF = cols.loc[index,"ECF"]
    Real_E = cols.loc[index,"Real_Effort"]    
    desired_UUCW = Real_E/((TCF*ECF)*20)-UAW
    cols.at[index, 'Desired_UUCW'] = np.round(desired_UUCW,1)
    
print(cols)

   UAW Smpl Avg Cmplx UUCW   TCF   ECF     UCP Real_Effort  Desired_UUCW
1    9    6  10    15  355  0.81  0.84   398.5        7970           576
2    8    4  20    15  445  0.99  0.99   398.1        7962           398
3    9    1   5    20  355  1.03   0.8  396.75        7935           472
4    8    5  10    15  350   0.9  0.91  390.25        7805           468
5    8    1  10    16  345   0.9  0.91   387.9        7758           465
..  ..  ...  ..   ...  ...   ...   ...     ...         ...           ...
67  18    2  18    18  460  1.05  0.95     290        5800           272
68  17    1  23    22  565  1.03   0.8  289.55        5791           334
69  19    5  30    10  475  0.95  0.92   289.1        5782           311
70   6    5  15     5  250     1  0.92   288.9        5778           308
71  18    5  18    17  460  0.85  0.89  288.75        5775           363

[71 rows x 10 columns]


In [16]:
def pso(function, e = 0.001, lr_reduce = False, minimum = True, lower_bound = 0, upper_bound = 10, n_particles = 10, iteration = 10, n_dimension = 1, c1 = 0.5, c2 = 0.5, w = 0.9, lower_teta = 0.4, upper_teta = 0.9, dif = False, dec_lr = False):
    
    #inisialisasi posisi dan kecepatan awal partikel
    particles_position = np.zeros((n_particles, n_dimension))
    particles_velocity = np.zeros((n_particles, n_dimension))
    
    for i in range(n_dimension):
        for j in range(n_particles):
            particles_position[j][i] = random.uniform(lower_bound, upper_bound)
            particles_velocity[j][i] = random.uniform(-abs(lower_bound-upper_bound),abs(lower_bound-upper_bound))
            
    #Menentukan Local Best dan Global Best awal
    local_value = []
    local_best = []
    for i in particles_position:
        value = function(i)
        local_value.append(value)
        local_best.append(i)
        
    #Global Best
    best_value = max(local_value)
    global_best = local_best[local_value.index(max(local_value))]
    
    #Update kecepatan dan posisi partikel
    it = 0
    stop = False
    while (it < iteration):
        if stop:
            break
        else:
            if lr_reduce:
                w = upper_teta - ((upper_teta - lower_teta)/iteration)*(it + 1)
            for i in range(len(particles_position)):
                new_particles_position = np.zeros((1,n_dimension))[0]
                for j in range(len(particles_position[i])):
                    r1 = random.uniform(0,1)
                    r2 = random.uniform(0,1)
                    velocity_new = w*particles_velocity[i][j] + c1*r1*(local_best[i][j] - particles_position[i][j]) + c2*r2*(global_best - particles_position[i][j])
                    position_new = velocity_new + particles_position[i][j]
                    new_particles_position[j] = position_new[0]
                    particles_velocity[i][j] = velocity_new[0]
                    
                #Update Local Best
                if minimum:
                    if function(new_particles_position) < function(local_best[i]):
                        local_best[i] = new_particles_position
                        local_value[i] = function(new_particles_position)
                else:
                    if function(new_particles_position) > function(local_best[i]):
                        local_best[i] = new_particles_position
                        local_value[i] = function(new_particles_position)
            
            #Update Global Best
            if minimum:
                best_value = min(local_value)
                global_best = local_best[local_value.index(min(local_value))]
            else:
                best_value = max(local_value)
                global_best = local_best[local_value.index(max(local_value))] 
                
        #print('global best', global_best)
        #print('value', best_value)
        #print('nilai w', w)
        
        if best_value < e:
            stop = True
        it += 1
        
        print('Global Best', global_best)
        print('Global Best Value', best_value)
        return global_best, best_value
